In [ ]:
import os
from IPython.display import display, Markdown

# Remove COLAB check and set root_dir to local directory
root_dir = "C:\jupyterlab\content\drive\lora"
deps_dir = os.path.join(root_dir, "deps")

project_name = "ProjectName" # Enter your project name here
project_name = project_name.strip()

folder_structure = "Organize by project (Lora/project_name/dataset)" # Choose your folder structure

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
  print("Please write a valid project_name.")
else:
  project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
  project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

  if "/Loras" in folder_structure:
    main_dir      = root_dir
    config_folder = os.path.join(main_dir, project_base)
    images_folder = os.path.join(main_dir, project_base, "dataset")
    if "/" in project_name:
      images_folder = os.path.join(images_folder, project_subfolder)
  else:
    main_dir      = root_dir
    config_folder = os.path.join(main_dir, "config", project_name)
    images_folder = os.path.join(main_dir, "datasets", project_name)

  for dir in [main_dir, deps_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ Project {project_name} is ready!")
  step1_installed_flag = True


In [ ]:
import os

if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

method = "Anime tags" 
tag_threshold = 0.35 
blacklist_tags = "virtual youtuber, official alternate costume, official alternate hairstyle, official alternate hair length, alternate costume, alternate hairstyle, alternate hair length, alternate hair color"
caption_min = 10 
caption_max = 75 

os.chdir(root_dir)
kohya = "kohya-trainer"
if not os.path.exists(kohya):
  os.system('git clone https://github.com/kohya-ss/sd-scripts kohya-trainer')
  os.chdir(kohya)
  os.system('git reset --hard 5050971ac687dca70ba0486a583d283e8ae324e2')
  os.chdir(root_dir)

if "tags" in method:
  if "step4a_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    os.system('pip install tensorflow==2.12.0 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6 torchvision albumentations')
    step4a_installed_flag = True

  print("\n🚶‍♂️ Launching program...\n")

  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  os.system(f'python {kohya}/finetune/tag_images_by_wd14_tagger.py {images_folder} --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 --model_dir={root_dir} --thresh={tag_threshold} --batch_size=8 --caption_extension=.txt --force_download')

  print("removing underscores and blacklist...")
  blacklisted_tags = [t.strip() for t in blacklist_tags.split(",")]
  from collections import Counter
  top_tags = Counter()
  for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
    with open(os.path.join(images_folder, txt), 'r') as f:
      tags = [t.strip() for t in f.read().split(",")]
      tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
      tags = [t for t in tags if t not in blacklisted_tags]
    top_tags.update(tags)
    with open(os.path.join(images_folder, txt), 'w') as f:
      f.write(", ".join(tags))

  print(f"📊 Tagging complete. Here are the top 50 tags in your dataset:")
  print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))

else: # Photos
  if "step4b_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    os.system('pip install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6')
    step4b_installed_flag = True

  print("\n🚶‍♂️ Launching program...\n")

  os.chdir(kohya)
  os.system(f'python {kohya}/finetune/make_captions.py {images_folder} --beam_search --max_data_loader_n_workers=2 --batch_size=8 --min_length={caption_min} --max_length={caption_max} --caption_extension=.txt')

  import random
  captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
  sample = []
  for txt in random.sample(captions, min(10, len(captions))):
    with open(os.path.join(images_folder, txt), 'r') as f:
      sample.append(f.read())

  os.chdir(root_dir)
  print(f"📊 Captioning complete. Here are {len(sample)} example captions from your dataset:")
  print("".join(sample))


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 5️⃣ Curate your tags
#@markdown Modify your dataset's tags. You can run this cell multiple times with different parameters. <p>

#@markdown Put an activation tag at the start of every text file. This is useful to make learning better and activate your Lora easier. Set `keep_tokens` to 1 when training.<p>
#@markdown Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.
global_activation_tag = "" #@param {type:"string"}
remove_tags = "" #@param {type:"string"}
#@markdown &nbsp;

#@markdown In this advanced section, you can search text files containing matching tags, and replace them with less/more/different tags. If you select the checkbox below, any extra tags will be put at the start of the file, letting you assign different activation tags to different parts of your dataset. Still, you may want a more advanced tool for this.
search_tags = "" #@param {type:"string"}
replace_with = "" #@param {type:"string"}
search_mode = "OR" #@param ["OR", "AND"]
new_becomes_activation_tag = False #@param {type:"boolean"}
#@markdown These may be useful sometimes. Will remove existing activation tags, be careful.
sort_alphabetically = False #@param {type:"boolean"}
remove_duplicates = False #@param {type:"boolean"}

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if global_activation_tag:
  print(f"\n📎 Applied new activation tag(s): {', '.join(activation_tag_list)}")
if remove_tags:
  print(f"\n🚮 Removed {remove_count} tags.")
if search_tags:
  print(f"\n💫 Replaced in {replace_count} files.")
print("\n✅ Done!")